In [1]:
import pandas as pd
#df is data frame mean the entire csv is parsed as csv
df=pd.read_csv("Data Preprocessing Sheet - data.csv")
df=df.drop_duplicates()
df["Name"]=df["Name"].str.strip().str.capitalize()
# we can create mapping for gender column since there are multiple values that need to be mapped with one value like Male:M MALE:M and so on
gender_mapping={
    "m":"M",
"f":"F",    
    "male":"M",

    "female":"F",
}
city_mapping={
    "new york":"New York",
    "ny":"New York",
    'newyork':"New York",
    "los angeles":"Los Angeles",
    "losangeles":"Los Angeles",
    "san francisco":"San Fransisco",
    "boston":"Boston",
    "chicago":"Chicago"
    ,"houston":"Houston",
    "hou":"Houston"
    ,"la":"Los Angeles",
    "sf":"San Francisco"

}
df["Gender"]=df["Gender"].str.lower().map(gender_mapping)
df["City"]=df["City"].str.lower().map(city_mapping)
df["Salary"]=df["Salary"].str.replace("$"," ").str.replace(",","")
med=df["Salary"].astype(float).median()
df["Salary"]=df["Salary"].astype(float).apply(lambda x:med if x<5000 else x)
age_median=df["Age"].astype(float).median()
def replace_with_zero(age):
 return 0 if pd.isna(age) else age
df["Age"]=df["Age"].apply(replace_with_zero)
df["Age"]=df["Age"].astype(float).apply(lambda x:age_median if x<0 or x>60 or x==0  else x)
df["Joining_Date"]=pd.to_datetime(df["Joining_Date"],errors="coerce")
df["Joining_Date"]=df["Joining_Date"].dt.strftime('%d-%m-%y')
median_date=pd.to_datetime(df["Joining_Date"]).median()
df["Joining_Date"]=df["Joining_Date"].fillna(median_date.strftime('%d-%m-%y'))
df["Height"]=df["Height"].fillna(df["Height"].median())
df=df.drop('Total_Sales_2025',axis=1)

df.to_csv("finaldata.csv")
print(df)

# print(df["Salary"].str.replace("$"," "))
# print(df.columns)



     ID     Name Gender   Age           City    Salary Joining_Date  Height  \
0   101    Alice      M  25.0       New York   50000.0     01-12-21    65.0   
1   102      Bob      M  30.5       New York   60000.0     30-09-17   170.0   
2   103  Charlie      F  45.0       New York   70000.0     15-07-20   180.0   
3   104     Dave      F  39.0    Los Angeles   80000.0     30-09-17    68.0   
4   105      Eve      F  30.5    Los Angeles   90000.0     30-09-17    72.0   
5   106    Frank      M  30.0  San Fransisco   10000.0     25-01-17   155.0   
6   107    Grace      M  29.0  San Francisco  120000.0     20-02-22    69.0   
7   108    Henry      F  27.0  San Fransisco   65000.0     10-03-23    73.0   
8   109   Isabel      F  30.5         Boston   55000.0     21-05-19   175.0   
9   110    Jason      M  37.0         Boston   45000.0     30-09-17    69.0   
10  111     Kate      F  30.5        Chicago   76000.0     30-09-17   162.0   
11  112      Leo      F  30.5        Chicago   85000

/var/folders/20/n6f48l15751dtp2km8bsrxvr0000gn/T/ipykernel_31860/1557483900.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  median_date=pd.to_datetime(df["Joining_Date"]).median()


In [3]:
import pandas as pd
import numpy as np

# Load and clean data
df2 = pd.read_csv("youtube_channel_info_final_updated.csv")
df2.drop_duplicates(inplace=True)

# Convert datatypes and handle zeroes
df2["view_count"] = df2["view_count"].astype(float)
df2["subscriber_count"] = df2["subscriber_count"].replace(0, 1).astype(float)
df2["video_count"] = df2["video_count"].astype(int)

# Replace too-low video counts with the median
med_videos = df2["video_count"].median()
df2["video_count"] = df2["video_count"].apply(lambda x: med_videos if x < med_videos else x)

# Compute ratios
df2["view_to_subscriber_ratio"] = (df2["subscriber_count"] / df2["view_count"]) * 100
df2["view_per_subscriber"] = df2["view_count"] / df2["subscriber_count"]

# Detect and cap outliers in view_per_subscriber (top 5%)
ratio_threshold = df2["view_per_subscriber"].quantile(0.95)
# The value below wh
# ich 95% of the data points lie.
# 	•	95% of your data values are less than or equal to this number.
	# •	5% of your data values are greater than this number.


In [4]:
# print(f"Outlier threshold (views per subscriber): {ratio_threshold:.2f}")

# Cap the extreme outliers

# 	1.	df2["view_per_subscriber"] > ratio_threshold
# → This creates a boolean mask (True/False for each row)
# → It checks which rows have view_per_subscriber greater than ratio_threshold.
# 	2.	df2.loc[mask, "subscriber_count"]
# → .loc is used to select rows and columns.
# → So, this targets the “subscriber_count” column only for those rows where the mask is True.
# 	3.	df2["view_count"] / ratio_threshold
# → This computes a new value: divides every view_count by ratio_threshold.
# 	4.	The assignment (=)
# → For rows where view_per_subscriber > ratio_threshold,
# the subscriber_count is replaced with the calculated value (view_count / ratio_threshold).
# df2.loc[df2["view_per_subscriber"] > ratio_threshold, "subscriber_count"] = \
#     df2["view_count"] / ratio_threshold

# Recalculate ratios after correction
df2["view_to_subscriber_ratio"] = (df2["subscriber_count"] / df2["view_count"]) * 100
df2["view_per_subscriber"] = df2["view_count"] / df2["subscriber_count"]

# Median ratio after correction
med_ratio = df2["view_to_subscriber_ratio"].median()
# print(f"Median view-to-subscriber ratio (after correction): {med_ratio:.4f}%")

# Set variable for median ratio (as a fraction, not %)
median_of_view_to_subscrber_ratio = med_ratio / 100
# print(f"The median of view-to-subscriber ratio is {median_of_view_to_subscrber_ratio:.4f}")

# Adjust very low subscriber counts (smooth correction)
df2['subscriber_count'] = np.where(
    df2['subscriber_count'] < median_of_view_to_subscrber_ratio * df2['view_count'],
    df2['subscriber_count'] * 1.2,  # modest boost instead of hard overwrite
    df2['subscriber_count']
)

# Final recomputation after all adjustments
df2["view_to_subscriber_ratio"] = (df2["subscriber_count"] / df2["view_count"]) * 100
df2["view_per_subscriber"] = df2["view_count"] / df2["subscriber_count"]

# Display a few rows to verify results
# print("\nSample data after refinement:")
# print(df2.head(10))

# import matplotlib.pyplot as plt

# plt.figure(figsize=(8,5))
# df2["view_per_subscriber"].hist(bins=50)
# plt.title("Distribution of Views per Subscriber (After Normalization)")
# plt.xlabel("Views per Subscriber")
# plt.ylabel("Frequency")
# plt.show()

df2.head(10)


,channel_name,channel_id,view_count,category,country,defaultLanguage,subscriber_count,created_date,description,custom_url,thumbnail,video_count,view_to_subscriber_ratio,view_per_subscriber
0,BLACKPINK,UCOmHUn--16B90oW2L6FRR3A,3.996259e+10,"https://en.wikipedia.org/wiki/Music_of_Asia, h...",KR,NaN,118800000.0,2016-06-29T03:15:23Z,BLACKPINK Official YouTube Channel 블랙핑크 공식 유튜브...,@blackpink,https://yt3.ggpht.com/U3VrCkKjzTpQ3VYv4SCPjNfD...,636.0,0.297278,336.385399
1,HYBE LABELS,UC3IZKseVpdzPSBaWxBxundA,4.160490e+10,"https://en.wikipedia.org/wiki/Hip_hop_music, h...",KR,NaN,94440000.0,2008-06-04T08:23:22Z,Welcome to the official YouTube channel of HYB...,@hybelabels,https://yt3.ggpht.com/ytc/AIdro_l8g0yRFG8xoe_q...,2817.0,0.226993,440.543169
2,music,UCh7X3QOvvDgBNB80_ILrl9Q,6.648800e+04,"https://en.wikipedia.org/wiki/Technology, http...",NaN,NaN,214.8,2017-11-17T05:53:19Z,NaN,@music-lw4ez,https://yt3.ggpht.com/ytc/AIdro_k9GtyIisxdF4aU...,510.0,0.323066,309.534451
3,#SPORTS,UCOtDOEFXsAHu0LTfK1Dejqw,3.118600e+04,"https://en.wikipedia.org/wiki/Action_game, htt...",NaN,NaN,44.4,2015-04-03T12:22:45Z,Welcome to one of the most beautiful channel o...,@sports-se7qn,https://yt3.ggpht.com/ytc/AIdro_lbN2EYYIH4TBth...,512.0,0.142372,702.387387
4,BILLIE EILISH,UCVNE660NcgYzi18LwwUZb7Q,1.431636e+07,NaN,NaN,NaN,82300.0,2019-01-18T05:14:32Z,NaN,@billieeilish8477,https://yt3.ggpht.com/ytc/AIdro_n3NfvUAVtcUJvW...,510.0,0.574867,173.953390
5,Shemaroo,UCF1JIbMUs6uqoZEY1Haw0GQ,2.701100e+10,"https://en.wikipedia.org/wiki/Entertainment, h...",IN,NaN,72480000.0,2007-09-01T11:44:51Z,"Welcome to ShemarooEnt, one of the finest dest...",@shemaroo,https://yt3.ggpht.com/ytc/AIdro_mhdF4RNZM9OHP-...,13374.0,0.268335,372.668381
6,JuegaGerman,UCYiGq8XF7YQD00x7wAd62Zg,1.757303e+10,https://en.wikipedia.org/wiki/Role-playing_vid...,CL,NaN,54200000.0,2013-05-19T00:09:13Z,Lento pero seguro.,@juegagerman,https://yt3.ggpht.com/vOsrLzWD4z1dbr470nEXydi3...,2368.0,0.308427,324.225618
7,LooLoo Kids - Nursery Rhymes and Children's Songs,UC4NALVCmcmL5ntpV0thoH6w,3.900036e+10,"https://en.wikipedia.org/wiki/Entertainment, h...",US,en,73200000.0,2014-08-05T20:15:33Z,LooLoo Kids💖 is an educational YouTube channel...,@loolookids,https://yt3.ggpht.com/XQ8Z5bwcV4dgjWEYFI1n1M9W...,1051.0,0.187691,532.791837
8,Tips Official,UCJrDMFOdv1I2k8n9oK_V21w,5.065703e+10,"https://en.wikipedia.org/wiki/Film, https://en...",IN,NaN,94920000.0,2007-05-22T10:13:28Z,The proud history of Tips Music Limited (Forme...,@tipsofficial,https://yt3.ggpht.com/zSQrmF-U4JwDv9QalTMOuEEW...,8639.0,0.187378,533.681305
9,MrBeast,UCX6OQ3DkcsbYNE6H8uQQuVA,9.670863e+10,https://en.wikipedia.org/wiki/Lifestyle_(socio...,US,en,443000000.0,2012-02-20T00:43:50Z,SUBSCRIBE FOR A COOKIE! New MrBeast or MrBeast...,@mrbeast,https://yt3.ggpht.com/nxYrc_1_2f77DoBadyxMTmv7...,906.0,0.458077,218.303895


In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zahranusrat/lifestyle-and-health-risk-prediction-dataset")

print("Path to dataset files:", path)
df=pd.read_csv("Lifestyle_and_Health_Risk_Prediction_Synthetic_Dataset.csv")
df.head(15)


/Users/samad/Documents/ML/mlenvironment/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/samad/.cache/kagglehub/datasets/zahranusrat/lifestyle-and-health-risk-prediction-dataset/versions/1


,age,weight,height,exercise,sleep,sugar_intake,smoking,alcohol,married,profession,bmi,health_risk
0,56,67,195,low,6.1,medium,yes,yes,yes,office_worker,17.6,high
1,69,76,170,high,6.9,high,no,no,no,teacher,26.3,high
2,46,106,153,high,6.6,low,yes,no,no,artist,45.3,high
3,32,54,186,medium,8.5,medium,no,no,no,artist,15.6,low
4,60,98,195,high,8.0,low,no,no,yes,teacher,25.8,high
5,25,96,160,medium,3.8,medium,no,no,yes,farmer,37.5,high
6,78,64,168,medium,9.9,high,no,no,yes,artist,22.7,high
7,38,76,194,low,6.6,medium,yes,no,yes,driver,20.2,high
8,56,58,158,medium,9.6,medium,no,no,yes,office_worker,23.2,low
9,75,94,147,none,8.1,medium,yes,no,no,farmer,43.5,high


In [29]:
import pandas as pd

df3 = pd.read_csv("fetched_dataset/movies.csv")

# Clean parentheses and replace weird dashes with a normal hyphen
df3['YEAR'] = (
    df3['YEAR']
    .astype(str)
    .str.strip("()")           # remove parentheses
    .str.replace('–', '-', regex=False)  # replace en dash with hyphen
)

def split_year_range(year_string):
    """Splits '2010-2021' or '2010-' or '2010' into start and end."""
    if not isinstance(year_string, str) or year_string.strip() == '':
        return None, None
    parts = year_string.split('-')
    start = parts[0].strip() if len(parts) > 0 and parts[0].strip().isdigit() else None
    end = parts[1].strip() if len(parts) > 1 and parts[1].strip().isdigit() else None
    if end is None:
        end = start  # same year if not provided
    return start, end

# Apply and expand manually
df3[['YEAR_STARTED', 'YEAR_ENDED']] = pd.DataFrame(
    df3['YEAR'].apply(lambda x: split_year_range(x)).tolist(),
    index=df3.index
)

# Convert to numeric safely
df3['YEAR_STARTED'] = pd.to_numeric(df3['YEAR_STARTED'], errors='coerce')
df3['YEAR_ENDED'] = pd.to_numeric(df3['YEAR_ENDED'], errors='coerce')

# print(df3[['YEAR', 'YEAR_STARTED', 'YEAR_ENDED']].head())
df3['GENRE'] = (
    df3['GENRE']
    .astype(str)
    .str.replace('\n', '', regex=False)
    .apply(lambda x: ', '.join([g.strip() for g in x.split(',') if g.strip() != '']))
)
df3.head()

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross,YEAR_STARTED,YEAR_ENDED
0,Blood Red Sky,2021,"Action, Horror, Thriller",6.1,\nA woman with a mysterious illness is forced ...,\n Director:\nPeter Thorwarth\n| \n Star...,"21,062",121.0,NaN,2021.0,2021.0
1,Masters of the Universe: Revelation,2021-,"Animation, Action, Adventure",5.0,\nThe war for Eternia begins again in what may...,"\n \n Stars:\nChris Wood, \nSara...","17,870",25.0,NaN,2021.0,2021.0
2,The Walking Dead,2010-2022,"Drama, Horror, Thriller",8.2,\nSheriff Deputy Rick Grimes wakes up from a c...,"\n \n Stars:\nAndrew Lincoln, \n...","885,805",44.0,NaN,2010.0,2022.0
3,Rick and Morty,2013-,"Animation, Adventure, Comedy",9.2,\nAn animated series that follows the exploits...,"\n \n Stars:\nJustin Roiland, \n...","414,849",23.0,NaN,2013.0,2013.0
4,Army of Thieves,2021,"Action, Crime, Horror",NaN,"\nA prequel, set before the events of Army of ...",\n Director:\nMatthias Schweighöfer\n| \n ...,NaN,NaN,NaN,2021.0,2021.0
